In [1]:
import pandas as pd
from sklearn.metrics import matthews_corrcoef
from tqdm import tqdm
import torch
from transformers import pipeline, AutoTokenizer
from sklearn.metrics import balanced_accuracy_score, precision_recall_fscore_support, accuracy_score, f1_score

C:\Users\mikeb\.conda\envs\sandbox\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
test = pd.read_csv('polnli_test_results.csv')

In [3]:
def metrics(df, preds, group_by=None):
    true_col = 'entailment'
    
    def get_metrics(y_true, y_pred):
        return {
            'MCC': matthews_corrcoef(y_true, y_pred),
            'Accuracy': accuracy_score(y_true, y_pred),
            'F1': f1_score(y_true, y_pred, average='weighted')
        }
    
    results = []
    
    if group_by not in ['dataset', 'task']:
        for col in preds:
            metrics = get_metrics(df[true_col], df[col])
            metrics['Column'] = col
            results.append(metrics)
    else:
        for col in preds:
            for group_name, group in df.groupby(group_by):
                metrics = get_metrics(group[true_col], group[col])
                metrics['Column'] = col
                metrics[group_by.capitalize()] = group_name
                results.append(metrics)
    
    results_df = pd.DataFrame(results)
    
    if group_by in ['dataset', 'task']:
        return results_df.set_index(['Column', group_by.capitalize()])
    else:
        return results_df.set_index('Column')

# Llama 3 8B

In [4]:
model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"

pipe = pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="cuda",
)

Loading checkpoint shards: 100%|██████████| 4/4 [00:42<00:00, 10.69s/it]


In [8]:
test['augmented_hypothesis'] = test['augmented_hypothesis'].str.lower()

In [9]:
user_message = """You are a classifier that can only respond with 0 or 1. I'm going to show you a short text sample and I want you to determine if {hypothesis}. Here is the text:
{doc}

If it is true that {hypothesis}, return 0. If it is not true that {hypothesis}, return 1.
Do not explain your answer, and only return 0 or 1.
"""

In [10]:
%%time
data = test
res = []

for i in data.index:
    doc = data.loc[i, 'premise']
    hypothesis = data.loc[i, 'augmented_hypothesis']
    messages = [
        {"role": "user", "content": user_message.format(doc = doc, hypothesis = hypothesis)},
    ]
    prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    outputs = pipe(prompt, max_new_tokens=2, do_sample=False, return_full_text = False, pad_token_id=pipe.tokenizer.eos_token_id, temperature = 0)
    res.extend(outputs)

res = [text['generated_text'] for text in res]
# return a list of unique responses from the model
print(set(res))

C:\Users\mikeb\.conda\envs\sandbox\Lib\site-packages\transformers\generation\configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
C:\Users\mikeb\.conda\envs\sandbox\Lib\site-packages\transformers\generation\configuration_utils.py:572: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
C:\Users\mikeb\.conda\envs\sandbox\Lib\site-packages\transformers\models\llama\modeling_llama.py:603: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
You seem to be using the pip

{'0', '1'}
CPU times: total: 24min 12s
Wall time: 25min 51s


In [ ]:
res = [num[0] for num in res]
test['llama'] = [1 if '1' in text else 0 for text in res]
test.to_csv('polnli_test_results.csv', index = False)

In [14]:
metrics(test, preds = ['base_polnli', 'base_nli', 'large_nli', 'llama'], group_by = None)

,MCC,Accuracy,F1
Column,,,
base_polnli,0.894269,0.948978,0.948852
base_nli,0.658454,0.834635,0.830205
large_nli,0.731021,0.869062,0.866256
llama,0.730997,0.862358,0.863467


In [15]:
metrics(test, preds = ['base_polnli', 'base_nli', 'large_nli', 'llama'], group_by = 'task')

MCC  Accuracy        F1
Column      Task                                                 
base_polnli event extraction         0.813742  0.906774  0.907042
            hatespeech and toxicity  0.841410  0.946036  0.945248
            stance detection         0.920600  0.961546  0.961488
            topic classification     0.926241  0.963834  0.963826
base_nli    event extraction         0.528918  0.746858  0.745710
            hatespeech and toxicity  0.494319  0.844437  0.822392
            stance detection         0.553824  0.786101  0.781703
            topic classification     0.875762  0.937653  0.937047
large_nli   event extraction         0.718805  0.850209  0.850508
            hatespeech and toxicity  0.571824  0.861426  0.854152
            stance detection         0.612922  0.811736  0.805470
            topic classification     0.899292  0.949634  0.949257
llama       event extraction         0.808244  0.905726  0.905480
            hatespeech and toxicity  0.559060  0.782145  0.799067
            stance detection         0.605609  0.798117  0.799651
            topic classification     0.918734  0.959396  0.959505